In [1]:
import cubo
import spyndex
import xarray as xr
from dask.distributed import Client
from dask.distributed import LocalCluster
import dask.dataframe as dd
import rioxarray
import geopandas as gpd
import pandas as pd
import planetary_computer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import os
from pathlib import Path
import numpy as np
import rioxarray as rxr
from glob import glob
import datetime
from shapely.geometry import Polygon
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import folium
import gc

# Filtered data to csv

In [2]:
def paths_to_datetimeindex(data_folder):
    dt_list = []
    for file_name in glob(str(data_folder) + "/*.tif"):
        dt = datetime.datetime.strptime(
            file_name.split("/")[-1].split("_")[2], "%Y%m%dT%H%M%S"
        )
        dt_list.append(dt)
    return pd.to_datetime(dt_list)


def create_cluster_blank_band(cluster_da: xr.DataArray, band_names: list[str] | str):
    cluster_da_mask = cluster_da
    # LOG.info("Creating cluster blank band")
    if type(band_names) is not list:
        band_list = [band_names]
    else:
        band_list = band_names

    for band_name in band_list:
        mask_aggs = []
        for x in cluster_da_mask:
            mask_aggs.append(xr.ones_like(x.sel(band="B02")))

        mask = xr.concat(mask_aggs, dim="time")

        mask.name = band_name
        cluster_da_mask = xr.concat(
            [
                cluster_da_mask,
                mask.expand_dims(band=[band_name]),
            ],
            dim="band",
        )
    return cluster_da_mask


def read_cluster_cube(data_folder: Path) -> xr.DataArray:
    time_var = xr.Variable("time", paths_to_datetimeindex(data_folder))
    x = []
    for i in glob(str(data_folder) + "/*.tif"):
        y = rxr.open_rasterio(i)
        y = y.assign_coords(id=i.split("/")[-1].split(".")[0], epsg=32756)  # type: ignore
        x.append(y)
        y.close()
    da = xr.concat(x, dim=time_var)
    da = da.sortby("time")  # type: ignore
    da = da.assign_coords(
        band=[
            "B02",  # blue
            "B03",  # green
            "B04",  # red
            "B08",  # NIR
            "SCL",
            "NDTI",
            "NDWI",
            "turbidity",
        ]
    )
    return da

## Read saved geotif

In [3]:
da_test = read_cluster_cube(Path("./data") / "cluster1")
da_test

<xarray.DataArray (time: 60, band: 8, y: 1024, x: 1024)> Size: 4GB
array([[[[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         ...,
         [ 1.34600000e-01,  1.28600000e-01,  1.27000000e-01, ...,
           1.24100000e-01,  1.25100000e-01,  1.23000000e-01],
         [ 1.17400000e-01,  1.18700000e-01,  1.20400000e-01, ...,
           1.25100000e-01,  1.25600000e-01,  1.23600000e-01],
         [ 1.15400000e-01,  1.18700000e-01,  1.16600000e-01, ...,
           1.25600000e-01,  1.23800000e-01,  1.26000000e-01]],

        [[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
...
         [            nan,             nan,             nan, ...,
          -4.10004100e-04,  2.75229358e-02,  2.50925545e-02],
         [            nan,             nan,             nan, ...,
           2.02228642e-02,  2.66001663e-02,  2.85477865e-02],
         [            nan,             nan,             nan, ...,
           1.65152766e-02,  1.99833472e-02,  1.90114068e-02]],

        [[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         ...,
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00]]]])
Coordinates:
  * x            (x) float64 8kB 5.378e+05 5.378e+05 ... 5.48e+05 5.48e+05
  * y            (y) float64 8kB 6.9e+06 6.9e+06 6.9e+06 ... 6.89e+06 6.89e+06
    spatial_ref  int64 8B 0
    id           (time) <U54 13kB 'S2B_MSIL2A_20230704T235239_R130_T56JNP_202...
    epsg         int64 8B 32756
  * time         (time) datetime64[ns] 480B 2023-07-04T23:52:39 ... 2025-10-0...
  * band         (band) <U9 288B 'B02' 'B03' 'B04' ... 'NDTI' 'NDWI' 'turbidity'
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      sentinel-2-l2a

## Filter by field data list

In [4]:
da_test = da_test.sel(
    band=[
        "B02",  # blue
        "B03",  # green
        "B04",  # red
        "B08",  # NIR
        "SCL",
        "NDTI",
        "NDWI",
    ]
)
da_test = create_cluster_blank_band(da_test, ["turbidity"])
da_test

/var/folders/tj/2681nd5j10b0nf0d9jlfsgrr0000gn/T/ipykernel_49500/3098747740.py:24: FutureWarning: In a future version of xarray the default value for coords will change from coords='different' to coords='minimal'. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set coords explicitly.
  mask = xr.concat(mask_aggs, dim="time")


<xarray.DataArray (time: 60, band: 8, y: 1024, x: 1024)> Size: 4GB
array([[[[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         ...,
         [ 1.34600000e-01,  1.28600000e-01,  1.27000000e-01, ...,
           1.24100000e-01,  1.25100000e-01,  1.23000000e-01],
         [ 1.17400000e-01,  1.18700000e-01,  1.20400000e-01, ...,
           1.25100000e-01,  1.25600000e-01,  1.23600000e-01],
         [ 1.15400000e-01,  1.18700000e-01,  1.16600000e-01, ...,
           1.25600000e-01,  1.23800000e-01,  1.26000000e-01]],

        [[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
...
         [            nan,             nan,             nan, ...,
          -4.10004100e-04,  2.75229358e-02,  2.50925545e-02],
         [            nan,             nan,             nan, ...,
           2.02228642e-02,  2.66001663e-02,  2.85477865e-02],
         [            nan,             nan,             nan, ...,
           1.65152766e-02,  1.99833472e-02,  1.90114068e-02]],

        [[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         ...,
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00]]]])
Coordinates:
  * x            (x) float64 8kB 5.378e+05 5.378e+05 ... 5.48e+05 5.48e+05
  * y            (y) float64 8kB 6.9e+06 6.9e+06 6.9e+06 ... 6.89e+06 6.89e+06
    spatial_ref  int64 8B 0
    id           (time) <U54 13kB 'S2B_MSIL2A_20230704T235239_R130_T56JNP_202...
    epsg         int64 8B 32756
  * time         (time) datetime64[ns] 480B 2023-07-04T23:52:39 ... 2025-10-0...
  * band         (band) object 64B 'B02' 'B03' 'B04' ... 'NDWI' 'turbidity'
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      sentinel-2-l2a

In [5]:
def filter_with_date_list(da, field_data, date_dif: int = 5):
    date_list = []
    date_dict = {}
    da_test = da.copy()
    i = 0
    for index, row in field_data.iterrows():
        selected_data = da_test.sel(time=row["date"], method="nearest")
        date_difference = (row["date"] - selected_data.time.to_pandas()).days
        if abs(date_difference) >= date_dif:
            i = i + 1
        else:
            if selected_data.time.to_pandas() not in date_list:
                date_list.append(selected_data.time.to_pandas())
                date_dict[selected_data.time.to_pandas()] = []

            sel_p = selected_data.sel(
                x=row.geometry.x, y=row.geometry.y, method="nearest"
            )
            date_dict[selected_data.time.to_pandas()].append(
                {
                    "sensor": row.sensor,
                    "turbidity": row.turbidity,
                    "location": sel_p,
                }
            )
    for index, row in date_dict.items():
        mask = xr.zeros_like(da_test.sel(band="B04")[-1])
        for turbid in row:
            selected_data = da_test.sel(time=index)
            mask.loc[dict(x=turbid["location"].x, y=turbid["location"].y)] = turbid[
                "turbidity"
            ]
            # mask.loc[dict(x=turbid["location"].x, y=turbid["location"].y)] = 2
            selected_data.loc[dict(band="turbidity")] = mask.values
            selected_data.loc[dict(band="turbidity")] = selected_data.sel(
                band="turbidity"
            ).where(selected_data.sel(band="turbidity") > 0)
            da_test.loc[dict(time=selected_data.time)] = selected_data

    da_test = da_test.sel(time=date_list)
    print(
        f"{i} data with more than 5 days difference, compared to {len(field_data)} total insitu data"
    )
    print(
        f"Loss of {(100 - ((len(field_data) - i) / len(field_data))* 100)}% in-situ data"
    )

    return da_test

In [6]:
sentinel_water_sensor_file = "./sentinel_water_quality.geojson"
sentinel_water = gpd.read_file(sentinel_water_sensor_file)
sentinel_water["date"] = pd.to_datetime(sentinel_water["date"])
sentinel_water.head()

,No,sensor,date,time,turbidity,geometry
0,1,DUC0001,2023-10-11,11.37.00,1.9,POINT (153.42031 -28.06907)
1,2,DUC0001,2024-01-25,13.01.00,6.9,POINT (153.42031 -28.06907)
2,3,DUC0001,2024-05-07,11.44.00,2.9,POINT (153.42031 -28.06907)
3,4,DUC0001,2024-07-19,10.39.00,1.2,POINT (153.42031 -28.06907)
4,5,DUC0001,2024-10-30,10.52.00,1.8,POINT (153.42031 -28.06907)


In [7]:
temp_data1 = da_test[-1].sel(band=["B04", "B03", "B02"])
temp_data1 = temp_data1.rio.write_crs(32756)
temp_data1 = temp_data1.rio.reproject(4326)
bbox = temp_data1.rio.bounds()
bbox1 = [(bbox[1], bbox[0]), (bbox[3], bbox[2])]

In [8]:
lats = temp_data1["y"].values
lons = temp_data1["x"].values
polygon1 = Polygon(
    [
        (lons.min(), lats.min()),
        (lons.min(), lats.max()),
        (lons.max(), lats.max()),
        (lons.max(), lats.min()),
    ]
)

In [9]:
def filter_within(field_data, polygon):
    filtered_data = field_data[field_data.within(polygon)]
    filtered_data = filtered_data.to_crs(32756)
    return filtered_data

In [10]:
sentinel_water_cluster1 = filter_within(sentinel_water, polygon1)
sentinel_water_cluster1.head()

,No,sensor,date,time,turbidity,geometry
0,1,DUC0001,2023-10-11,11.37.00,1.9,POINT (541299.454 6895075.477)
1,2,DUC0001,2024-01-25,13.01.00,6.9,POINT (541299.454 6895075.477)
2,3,DUC0001,2024-05-07,11.44.00,2.9,POINT (541299.454 6895075.477)
3,4,DUC0001,2024-07-19,10.39.00,1.2,POINT (541299.454 6895075.477)
4,5,DUC0001,2024-10-30,10.52.00,1.8,POINT (541299.454 6895075.477)


In [11]:
da_test = filter_with_date_list(da_test, sentinel_water_cluster1)
da_test

20 data with more than 5 days difference, compared to 40 total insitu data
Loss of 50.0% in-situ data


<xarray.DataArray (time: 8, band: 8, y: 1024, x: 1024)> Size: 537MB
array([[[[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         ...,
         [ 0.1299    ,  0.1304    ,  0.1287    , ...,  0.144     ,
           0.1417    ,  0.1395    ],
         [ 0.1226    ,  0.1234    ,  0.1246    , ...,  0.1396    ,
           0.1412    ,  0.1406    ],
         [ 0.1251    ,  0.1253    ,  0.1282    , ...,  0.1355    ,
           0.133     ,  0.1393    ]],

        [[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
...
         [        nan,         nan,         nan, ...,  0.01533071,
           0.02204586,  0.02525476],
         [        nan,         nan,         nan, ...,  0.01445466,
           0.02051506,  0.02239789],
         [        nan,         nan,         nan, ...,  0.0193662 ,
           0.02411223,  0.02280702]],

        [[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         ...,
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]]]])
Coordinates:
  * x            (x) float64 8kB 5.378e+05 5.378e+05 ... 5.48e+05 5.48e+05
  * y            (y) float64 8kB 6.9e+06 6.9e+06 6.9e+06 ... 6.89e+06 6.89e+06
    spatial_ref  int64 8B 0
    id           (time) <U54 2kB 'S2B_MSIL2A_20231012T235229_R130_T56JNP_2023...
    epsg         int64 8B 32756
  * time         (time) datetime64[ns] 64B 2023-10-12T23:52:29 ... 2024-09-21...
  * band         (band) object 64B 'B02' 'B03' 'B04' ... 'NDWI' 'turbidity'
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      sentinel-2-l2a

In [12]:
download_location = Path("./data")
year_list = [2023, 2024, 2025]

pd_dataframe = pd.DataFrame()

for year in year_list:
    print(f"Loading year: {year}")
    da = da_test.sel(time=da_test.time.dt.year.isin(year))
    print("Creating dataframe")
    df = da.to_dataframe(name="s2")
    da_pivot = pd.pivot_table(
        data=df,
        index=["time", "x", "y"],
        columns=["band"],
        values=["s2"],
    )
    da_pivot.columns = [a[1] for a in da_pivot.columns.to_flat_index()]
    da_pivot = da_pivot.reset_index()
    pd_dataframe = pd.concat([pd_dataframe, da_pivot])
pd_dataframe

Loading year: 2023
Creating dataframe
Loading year: 2024
Creating dataframe
Loading year: 2025
Creating dataframe


,time,x,y,B02,B03,B04,B08,NDTI,NDWI,SCL,turbidity
0,2023-10-02 23:52:29,537750.0,6889860.0,0.1295,0.1243,0.1214,0.1916,NaN,NaN,3.0,NaN
1,2023-10-02 23:52:29,537750.0,6889870.0,0.1294,0.1246,0.1202,0.1910,NaN,NaN,3.0,NaN
2,2023-10-02 23:52:29,537750.0,6889880.0,0.1298,0.1264,0.1237,0.2082,NaN,NaN,3.0,NaN
3,2023-10-02 23:52:29,537750.0,6889890.0,0.1295,0.1284,0.1236,0.2048,NaN,NaN,3.0,NaN
4,2023-10-02 23:52:29,537750.0,6889900.0,0.1305,0.1268,0.1228,0.1958,NaN,NaN,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2086907,2025-07-13 23:52:39,547980.0,6900000.0,0.1190,0.1096,0.1011,0.0999,-0.040342,0.046301,6.0,NaN
2086908,2025-07-13 23:52:39,547980.0,6900010.0,0.1173,0.1095,0.1007,0.1008,-0.041865,0.041369,6.0,NaN
2086909,2025-07-13 23:52:39,547980.0,6900020.0,0.1176,0.1089,0.1007,0.1009,-0.039122,0.038132,6.0,NaN
2086910,2025-07-13 23:52:39,547980.0,6900030.0,0.1204,0.1083,0.1012,0.1014,-0.033890,0.032904,6.0,NaN


In [13]:
pd_dataframe.to_csv(download_location / "filtered_cluster1.csv")

In [14]:
da_test2 = read_cluster_cube(Path("./data") / "cluster2")
da_test2

<xarray.DataArray (time: 71, band: 8, y: 1024, x: 1024)> Size: 5GB
array([[[[ 1.80200000e-01,  1.54800000e-01,  1.47300000e-01, ...,
           1.11100000e-01,  1.12300000e-01,  1.12000000e-01],
         [ 2.93800000e-01,  2.45600000e-01,  2.27200000e-01, ...,
           1.12100000e-01,  1.12100000e-01,  1.13300000e-01],
         [ 3.67800000e-01,  4.05400000e-01,  3.81000000e-01, ...,
           1.14200000e-01,  1.12600000e-01,  1.14400000e-01],
         ...,
         [            nan,             nan,             nan, ...,
           1.15700000e-01,  1.11300000e-01,  1.11700000e-01],
         [            nan,             nan,             nan, ...,
           1.13600000e-01,  1.12600000e-01,  1.10600000e-01],
         [            nan,             nan,             nan, ...,
           1.15700000e-01,  1.13800000e-01,  1.10600000e-01]],

        [[ 1.99200000e-01,  1.74200000e-01,  1.71400000e-01, ...,
           1.00500000e-01,  1.00100000e-01,  1.01900000e-01],
         [ 3.06200000e-01,  2.66600000e-01,  2.50400000e-01, ...,
           1.02100000e-01,  1.02800000e-01,  9.95000000e-02],
         [ 3.79800000e-01,  4.21600000e-01,  3.83200000e-01, ...,
           1.01500000e-01,  1.00100000e-01,  1.01800000e-01],
...
         [ 7.62081784e-02,  1.02742910e-01,  9.83981693e-02, ...,
           1.76041219e-02,  1.60103851e-02,  1.89845475e-02],
         [ 7.18722272e-02,  8.19525302e-02,  1.07776262e-01, ...,
           2.46007769e-02,  1.20898100e-02,  3.06882946e-03],
         [ 7.23055935e-02,  4.50184502e-02,             nan, ...,
           2.01224847e-02,  3.84779820e-03,  8.43170320e-04]],

        [[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         ...,
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00]]]])
Coordinates:
  * x            (x) float64 8kB 5.378e+05 5.378e+05 ... 5.48e+05 5.48e+05
  * y            (y) float64 8kB 6.911e+06 6.911e+06 ... 6.901e+06 6.901e+06
    spatial_ref  int64 8B 0
    id           (time) <U54 15kB 'S2B_MSIL2A_20230704T235239_R130_T56JNQ_202...
    epsg         int64 8B 32756
  * time         (time) datetime64[ns] 568B 2023-07-04T23:52:39 ... 2025-10-0...
  * band         (band) <U9 288B 'B02' 'B03' 'B04' ... 'NDTI' 'NDWI' 'turbidity'
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      sentinel-2-l2a

In [15]:
da_test2 = da_test2.sel(
    band=[
        "B02",  # blue
        "B03",  # green
        "B04",  # red
        "B08",  # NIR
        "SCL",
        "NDTI",
        "NDWI",
    ]
)
da_test2 = create_cluster_blank_band(da_test2, ["turbidity"])
da_test2

/var/folders/tj/2681nd5j10b0nf0d9jlfsgrr0000gn/T/ipykernel_49500/3098747740.py:24: FutureWarning: In a future version of xarray the default value for coords will change from coords='different' to coords='minimal'. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set coords explicitly.
  mask = xr.concat(mask_aggs, dim="time")


<xarray.DataArray (time: 71, band: 8, y: 1024, x: 1024)> Size: 5GB
array([[[[ 1.80200000e-01,  1.54800000e-01,  1.47300000e-01, ...,
           1.11100000e-01,  1.12300000e-01,  1.12000000e-01],
         [ 2.93800000e-01,  2.45600000e-01,  2.27200000e-01, ...,
           1.12100000e-01,  1.12100000e-01,  1.13300000e-01],
         [ 3.67800000e-01,  4.05400000e-01,  3.81000000e-01, ...,
           1.14200000e-01,  1.12600000e-01,  1.14400000e-01],
         ...,
         [            nan,             nan,             nan, ...,
           1.15700000e-01,  1.11300000e-01,  1.11700000e-01],
         [            nan,             nan,             nan, ...,
           1.13600000e-01,  1.12600000e-01,  1.10600000e-01],
         [            nan,             nan,             nan, ...,
           1.15700000e-01,  1.13800000e-01,  1.10600000e-01]],

        [[ 1.99200000e-01,  1.74200000e-01,  1.71400000e-01, ...,
           1.00500000e-01,  1.00100000e-01,  1.01900000e-01],
         [ 3.06200000e-01,  2.66600000e-01,  2.50400000e-01, ...,
           1.02100000e-01,  1.02800000e-01,  9.95000000e-02],
         [ 3.79800000e-01,  4.21600000e-01,  3.83200000e-01, ...,
           1.01500000e-01,  1.00100000e-01,  1.01800000e-01],
...
         [ 7.62081784e-02,  1.02742910e-01,  9.83981693e-02, ...,
           1.76041219e-02,  1.60103851e-02,  1.89845475e-02],
         [ 7.18722272e-02,  8.19525302e-02,  1.07776262e-01, ...,
           2.46007769e-02,  1.20898100e-02,  3.06882946e-03],
         [ 7.23055935e-02,  4.50184502e-02,             nan, ...,
           2.01224847e-02,  3.84779820e-03,  8.43170320e-04]],

        [[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         ...,
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
         [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00]]]])
Coordinates:
  * x            (x) float64 8kB 5.378e+05 5.378e+05 ... 5.48e+05 5.48e+05
  * y            (y) float64 8kB 6.911e+06 6.911e+06 ... 6.901e+06 6.901e+06
    spatial_ref  int64 8B 0
    id           (time) <U54 15kB 'S2B_MSIL2A_20230704T235239_R130_T56JNQ_202...
    epsg         int64 8B 32756
  * time         (time) datetime64[ns] 568B 2023-07-04T23:52:39 ... 2025-10-0...
  * band         (band) object 64B 'B02' 'B03' 'B04' ... 'NDWI' 'turbidity'
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      sentinel-2-l2a

In [16]:
temp_data2 = da_test2[-1].sel(band=["B04", "B03", "B02"])
temp_data2 = temp_data2.rio.write_crs(32756)
temp_data2 = temp_data2.rio.reproject(4326)
bbox = temp_data1.rio.bounds()
bbox2 = [(bbox[1], bbox[0]), (bbox[3], bbox[2])]

In [17]:
lats = temp_data2["y"].values
lons = temp_data2["x"].values
polygon2 = Polygon(
    [
        (lons.min(), lats.min()),
        (lons.min(), lats.max()),
        (lons.max(), lats.max()),
        (lons.max(), lats.min()),
    ]
)

In [18]:
sentinel_water_cluster2 = filter_within(sentinel_water, polygon2)
sentinel_water_cluster2.head()

,No,sensor,date,time,turbidity,geometry
40,41,BWC0001,2023-10-11,10.07.00,1.70,POINT (540521.17 6902786.806)
41,42,BWC0001,2024-01-25,11.10.00,4.10,POINT (540521.17 6902786.806)
42,43,BWC0001,2024-05-07,10.33.00,2.20,POINT (540521.17 6902786.806)
43,44,BWC0001,2025-07-19,09.32.00,0.81,POINT (540521.17 6902786.806)
44,45,BWC0001,2024-10-30,09.44.00,1.80,POINT (540521.17 6902786.806)


In [19]:
da_test2 = filter_with_date_list(da_test2, sentinel_water_cluster2)
da_test2

11 data with more than 5 days difference, compared to 24 total insitu data
Loss of 45.833333333333336% in-situ data


<xarray.DataArray (time: 9, band: 8, y: 1024, x: 1024)> Size: 604MB
array([[[[ 0.1574    ,  0.1536    ,  0.1466    , ...,  0.1376    ,
           0.1343    ,  0.1306    ],
         [ 0.2842    ,  0.2494    ,  0.2208    , ...,  0.1336    ,
           0.1413    ,  0.1371    ],
         [ 0.44      ,  0.4444    ,  0.4032    , ...,  0.1285    ,
           0.128     ,  0.1316    ],
         ...,
         [ 0.1255    ,  0.123     ,  0.1247    , ...,  0.1331    ,
           0.1321    ,  0.1358    ],
         [ 0.1369    ,  0.1339    ,  0.1308    , ...,  0.1342    ,
           0.131     ,  0.1303    ],
         [ 0.165     ,  0.1618    ,  0.1499    , ...,  0.137     ,
           0.1331    ,  0.1301    ]],

        [[ 0.1802    ,  0.1679    ,  0.1592    , ...,  0.1266    ,
           0.1266    ,  0.1148    ],
         [ 0.3032    ,  0.2626    ,  0.241     , ...,  0.125     ,
           0.1302    ,  0.1221    ],
         [ 0.4508    ,  0.4396    ,  0.4168    , ...,  0.119     ,
           0.1234    ,  0.1245    ],
...
         [ 0.05015002,  0.0627687 ,  0.0625543 , ...,  0.05247954,
           0.05124451,  0.05212355],
         [ 0.04734219,  0.04477612,  0.01356064, ...,  0.05181598,
           0.06329114,  0.0620229 ],
         [-0.04994761,         nan,         nan, ...,  0.04585366,
           0.05176584,  0.03976722]],

        [[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         ...,
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]]]])
Coordinates:
  * x            (x) float64 8kB 5.378e+05 5.378e+05 ... 5.48e+05 5.48e+05
  * y            (y) float64 8kB 6.911e+06 6.911e+06 ... 6.901e+06 6.901e+06
    spatial_ref  int64 8B 0
    id           (time) <U54 2kB 'S2B_MSIL2A_20231012T235229_R130_T56JNQ_2023...
    epsg         int64 8B 32756
  * time         (time) datetime64[ns] 72B 2023-10-12T23:52:29 ... 2025-07-03...
  * band         (band) object 64B 'B02' 'B03' 'B04' ... 'NDWI' 'turbidity'
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      sentinel-2-l2a

In [20]:
download_location = Path("./data")
year_list = [2023, 2024, 2025]

pd_dataframe2 = pd.DataFrame()

for year in year_list:
    print(f"Loading year: {year}")
    da = da_test2.sel(time=da_test2.time.dt.year.isin(year))
    print("Creating dataframe")
    df = da.to_dataframe(name="s2")
    da_pivot = pd.pivot_table(
        data=df,
        index=["time", "x", "y"],
        columns=["band"],
        values=["s2"],
    )
    da_pivot.columns = [a[1] for a in da_pivot.columns.to_flat_index()]
    da_pivot = da_pivot.reset_index()
    pd_dataframe2 = pd.concat([pd_dataframe, da_pivot])
pd_dataframe2

Loading year: 2023
Creating dataframe
Loading year: 2024
Creating dataframe
Loading year: 2025
Creating dataframe


,time,x,y,B02,B03,B04,B08,NDTI,NDWI,SCL,turbidity
0,2023-10-02 23:52:29,537750.0,6889860.0,0.1295,0.1243,0.1214,0.1916,NaN,NaN,3.0,NaN
1,2023-10-02 23:52:29,537750.0,6889870.0,0.1294,0.1246,0.1202,0.1910,NaN,NaN,3.0,NaN
2,2023-10-02 23:52:29,537750.0,6889880.0,0.1298,0.1264,0.1237,0.2082,NaN,NaN,3.0,NaN
3,2023-10-02 23:52:29,537750.0,6889890.0,0.1295,0.1284,0.1236,0.2048,NaN,NaN,3.0,NaN
4,2023-10-02 23:52:29,537750.0,6889900.0,0.1305,0.1268,0.1228,0.1958,NaN,NaN,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3145723,2025-07-18 23:53:11,548020.0,6911120.0,0.1093,0.1051,0.0995,0.0994,-0.027370,0.027873,6.0,NaN
3145724,2025-07-18 23:53:11,548020.0,6911130.0,0.1095,0.1066,0.0992,0.0996,-0.035957,0.033948,6.0,NaN
3145725,2025-07-18 23:53:11,548020.0,6911140.0,0.1118,0.1057,0.0991,0.0985,-0.032227,0.035260,6.0,NaN
3145726,2025-07-18 23:53:11,548020.0,6911150.0,0.1073,0.1037,0.0981,0.0979,-0.027750,0.028770,6.0,NaN


In [21]:
pd_dataframe2.to_csv(download_location / "filtered_cluster2.csv")

In [22]:
pd_dataframe3 = pd.concat([pd_dataframe, pd_dataframe2])
pd_dataframe3.to_csv(download_location / "filtered_data.csv")

In [23]:
del da_test
del da_test2
del pd_dataframe
del pd_dataframe2
del pd_dataframe3
gc.collect()

0

# Unfiltered data to csv

In [ ]:
def filter_with_date_list2(da, field_data, date_dif: int = 5):
    date_list = []
    date_dict = {}
    da_test = da.copy()
    i = 0
    for index, row in field_data.iterrows():
        selected_data = da_test.sel(time=row["date"], method="nearest")
        date_difference = (row["date"] - selected_data.time.to_pandas()).days
        if abs(date_difference) >= date_dif:
            i = i + 1
        else:
            if selected_data.time.to_pandas() not in date_list:
                date_list.append(selected_data.time.to_pandas())
                date_dict[selected_data.time.to_pandas()] = []

            sel_p = selected_data.sel(
                x=row.geometry.x, y=row.geometry.y, method="nearest"
            )
            date_dict[selected_data.time.to_pandas()].append(
                {
                    "sensor": row.sensor,
                    "turbidity": row.turbidity,
                    "location": sel_p,
                }
            )
    for index, row in date_dict.items():
        mask = xr.zeros_like(da_test.sel(band="B04")[-1])
        for turbid in row:
            selected_data = da_test.sel(time=index)
            mask.loc[dict(x=turbid["location"].x, y=turbid["location"].y)] = turbid[
                "turbidity"
            ]
            # mask.loc[dict(x=turbid["location"].x, y=turbid["location"].y)] = 2
            selected_data.loc[dict(band="turbidity")] = mask.values
            selected_data.loc[dict(band="turbidity")] = selected_data.sel(
                band="turbidity"
            ).where(selected_data.sel(band="turbidity") > 0)
            da_test.loc[dict(time=selected_data.time)] = selected_data

    print(
        f"{i} data with more than 5 days difference, compared to {len(field_data)} total insitu data"
    )
    print(
        f"Loss of {(100 - ((len(field_data) - i) / len(field_data))* 100)}% in-situ data"
    )

    return da_test

In [ ]:
da_test = read_cluster_cube(Path("./data") / "cluster1")
da_test

In [ ]:
da_test = filter_with_date_list2(da_test, sentinel_water_cluster1)
da_test

In [ ]:
download_location = Path("./data")
year_list = [2023, 2024, 2025]

pd_dataframe = pd.DataFrame()

for year in year_list:
    print(f"Loading year: {year}")
    da = da_test.sel(time=da_test.time.dt.year.isin(year), band=["NDTI", "turbidity"])
    print("Creating dataframe")
    df = da.to_dataframe(name="s2")
    da_pivot = pd.pivot_table(
        data=df,
        index=["time", "x", "y"],
        columns=["band"],
        values=["s2"],
    )
    da_pivot.columns = [a[1] for a in da_pivot.columns.to_flat_index()]
    da_pivot = da_pivot.reset_index()
    pd_dataframe = pd.concat([pd_dataframe, da_pivot])
pd_dataframe

: 

: 

In [ ]:
pd_dataframe.to_csv(download_location / "cluster1.csv")

In [ ]:
da_test2 = read_cluster_cube(Path("./data") / "cluster2")
da_test2

In [ ]:
da_test2 = filter_with_date_list2(da_test2, sentinel_water_cluster2)
da_test2

In [ ]:
download_location = Path("./data")
year_list = [2023, 2024, 2025]

pd_dataframe2 = pd.DataFrame()

for year in year_list:
    print(f"Loading year: {year}")
    da = da_test2.sel(time=da_test2.time.dt.year.isin(year), band=["NDTI", "turbidity"])
    print("Creating dataframe")
    df = da.to_dataframe(name="s2")
    da_pivot = pd.pivot_table(
        data=df,
        index=["time", "x", "y"],
        columns=["band"],
        values=["s2"],
    )
    da_pivot.columns = [a[1] for a in da_pivot.columns.to_flat_index()]
    da_pivot = da_pivot.reset_index()
    pd_dataframe2 = pd.concat([pd_dataframe2, da_pivot])
pd_dataframe2

In [ ]:
pd_dataframe2.to_csv(download_location / "cluster2.csv")

In [ ]:
pd_dataframe3 = pd.concat([pd_dataframe, pd_dataframe2])
pd_dataframe3.to_csv(download_location / "all_cluster_data.csv")